In [2]:
#import modules
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pylab as plt
import seaborn as sns
import credentials
import datetime
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import unittest

In [3]:
#import database credentials
URI = "dublinbikeappdb.cxaxe40vwlui.us-east-1.rds.amazonaws.com"
DB = "dbikes1"
name = credentials.name
pw = credentials.password
engine = create_engine("mysql+mysqlconnector://{}:{}@{}:3306/{}".format(name,pw,URI,DB),echo=False)

In [6]:
class testCase(unittest.TestCase):
    #test accessibility to database
    def test_database_access_stations(self):
        #test that the database can connect and that the static stations table is functioning
        """Initate connection"""
        select = """SELECT *
        FROM stations s"""
    
        dfs = pd.read_sql_query(select, engine)
        self.assertEqual(dfs.shape[0],109)
        self.assertEqual(dfs.shape[1],6)
        
    def test_database_access_weather(self):
        #test that the database can connect and that the static stations table is functioning
        weather = '''SELECT *
        FROM weather'''
        dfw = dfs = pd.read_sql_query(weather, engine)
        self.assertEqual(dfw.shape[1],13)
        
    def test_database_access_availability(self):
        #test that the database can connect and that the static stations table is functioning
        availability = '''SELECT a.number
        FROM availability a
        GROUP BY a.number
        '''

        dfa = pd.read_sql_query(availability, engine)
        self.assertEqual(dfa.shape[0],110)
        #NOTE: This station number differs to the previous station number as it includes an experimental station
        #We were under the assumption that this station is a prototype and different to the other stations
        #We withheld this station from the stations table until it is out of prototype stage, as it is signigicantly
        #different to the type of bikes offered at the other stations, and customers looking for a none electric bike
        #may become frustrated if they wish to rent a normal bike and find a station only has electric. Thus, this 
        #station has been excluded from the models, but data is still being collected on the station in the event that
        #we wish to include the station for futher reference
        
    def test_pickle(self):
        #test to make sure that both pickle file is accessible and the models on the file are functioning
        import pickle
        pickle_in = open("models.pkl", "rb")
        models = pickle.load(pickle_in)
        model = models[2][0]
    
        #compute bikes available at this time
        dic = {'day': 5, 'humid': 61, 'temp': 9.53}
        result = model.predict([np.array([16, dic['day'], dic['humid'], dic['temp']])])
        result = result[0]

        #ensures prediction cannot be negitive
        if result <= 0:
            result = 0
        
        self.assertEqual(int(result),5)

unittest.main(argv=[''],verbosity=2, exit=False)

test_database_access_availability (__main__.testCase) ... ok
test_database_access_stations (__main__.testCase)
Initate connection ... ok
test_database_access_weather (__main__.testCase) ... ok
test_pickle (__main__.testCase) ... C:\ProgramData\Anaconda3\lib\unittest\case.py:633: ResourceWarning: unclosed file <_io.BufferedReader name='models.pkl'>
  method()
ok

----------------------------------------------------------------------
Ran 4 tests in 4.419s

OK
